In [1]:
import sys
import math
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from torch.cuda.amp import autocast
import torch
import torch
import matplotlib.pyplot as plt
from datasets import Dataset
from collections import defaultdict
from torch.utils.data import DataLoader
from transformers import LlamaTokenizer, LlamaForCausalLM
import json
from transformers import LlamaConfig
from einops import rearrange, einsum
from tqdm import tqdm
import sys
import os
sys.path.insert(0, '/home/jnainani_umass_edu/codellm/MechInterpCodeLLMs/finetuneMI')
print("Current Working Directory:", os.getcwd())
from data.data_utils import *
from baukit.baukit import nethook
from baukit.baukit.nethook import TraceDict
from experiment_1.pp_utils import compute_prev_query_box_pos
print("here")
# pretrained_model_dir = os.path.abspath("../../pretrained_models/llama/llama-2-7b")
# print("LLAMA 2 Weights: ", pretrained_model_dir)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device is: ", device)

/home/jnainani_umass_edu/.conda/envs/finetuning/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/jnainani_umass_edu/.conda/envs/finetuning/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/jnainani_umass_edu/.conda/envs/finetuning/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Current Working Directory: /home/jnainani_umass_edu/codellm/MechInterpCodeLLMs/finetuneMI/experiment_1
here
Device is:  cuda


In [2]:
%cd ..

/home/jnainani_umass_edu/codellm/MechInterpCodeLLMs/finetuneMI


In [3]:
%pwd

'/home/jnainani_umass_edu/codellm/MechInterpCodeLLMs/finetuneMI'

In [4]:
seed = 5
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# tokenizer = LlamaTokenizer.from_pretrained(
#     "hf-internal-testing/llama-tokenizer", padding_side="right"
# )
# tokenizer.pad_token_id = tokenizer.eos_token_id


In [5]:
# dataloader = DataLoader(dataset, batch_size=batch_size)
# with open('/home/jnainani_umass_edu/codellm/pretrained_models/llama/llama-2-7b/params.json', 'r') as f:
#     config_dict = json.load(f)
# config = LlamaConfig(**config_dict)

# model = LlamaForCausalLM(config)
# model_weights = torch.load('/home/jnainani_umass_edu/codellm/pretrained_models/llama/llama-2-7b/consolidated.00.pth', map_location=device)
# model.load_state_dict(model_weights)

# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
# model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")


path = "huggyllama/llama-7b"

model = LlamaForCausalLM.from_pretrained(path).to(device)
tokenizer = LlamaTokenizer.from_pretrained(
    "hf-internal-testing/llama-tokenizer", padding_side="right"
)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# model = torch.nn.DataParallel(model).to(device)  # Data parallelism
model.eval()

dummy_text = ["The document is in Box X, the pot is in Box T, the magnet is in Box A. Box X contains the ",
              "The document is in Box X, the pot is in Box T, the magnet is in Box A. Box T contains the ", 
              "The document is in Box X, the pot is in Box T, the magnet is in Box A. Box A contains the "]

for dumb in dummy_text:
    input_ids = tokenizer.encode(dumb, return_tensors="pt")
    model.eval()

    # If you are using a GPU, move the input ids to the GPU
    if torch.cuda.is_available():
        input_ids = input_ids.to('cuda')
        model = model.to('cuda')

    # Perform inference
    model.eval()
    with torch.no_grad():
        with autocast():  # Enable mixed precision
            outputs = model.generate(input_ids, max_length=55)

    # Decode predictions
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Predicted Text:", predicted_text)

In [6]:
from pp_utils import (
    get_model_and_tokenizer,
    load_dataloader,
    get_caches,
    compute_topk_components,
    patching_receiver_heads,
    patching_sender_heads,
    get_receiver_layers,
)

In [7]:
from experiment_1.path_patching import apply_pp

In [8]:
%pwd

'/home/jnainani_umass_edu/codellm/MechInterpCodeLLMs/finetuneMI'

In [9]:
import transformers

datafile= "data/dataset.jsonl"
num_boxes= 7

num_samples= 30
n_value_fetcher= 20  # Goat / FLoat circuit: 50, Llama circuit: 20
n_pos_trans= 5  # Goat / FLoat circuit: 20, Llama circuit: 5
n_pos_detect = 10  # Goat / FLoat circuit: 30, Llama circuit: 10
n_struct_read = 5  # Goat / FLoat circuit: 5, Llama circuit: 5
output_path = "experiment_1/results1/path_patching/"
seed = 20  # Goat circuit: 82, FLoat circuit: 85, Llama circuit: 20
batch_size = 100

print(f"DATAFILE: {datafile}")
print(f"NUM BOXES: {num_boxes}")
print(f"NUM SAMPLES: {num_samples}")
print(f"VALUE FETCHER HEADS: {n_value_fetcher}")
print(f"POSITION TRANSMITTER HEADS: {n_pos_trans}")
print(f"POSITION DETECTOR HEADS: {n_pos_detect}")
print(f"STRUCTURAL READER HEADS: {n_struct_read}")
print(f"OUTPUT PATH: {output_path}")
print(f"SEED: {seed}")
print(f"BATCH SIZE: {batch_size}\n")

def set_seed(seed: int):
    """
    Sets the seed for reproducibility.

    Args:
        seed (int): Seed to use.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    transformers.set_seed(seed)
    
set_seed(seed)

dataloader = load_dataloader(
        model=model,
        tokenizer=tokenizer,
        datafile=datafile,
        num_samples=num_samples,
        num_boxes=num_boxes,
        batch_size=batch_size,
    )

DATAFILE: data/dataset.jsonl
NUM BOXES: 7
NUM SAMPLES: 30
VALUE FETCHER HEADS: 20
POSITION TRANSMITTER HEADS: 5
POSITION DETECTOR HEADS: 10
STRUCTURAL READER HEADS: 5
OUTPUT PATH: experiment_1/results1/path_patching/
SEED: 20
BATCH SIZE: 100



## Step 1: Compute clean and corrupted cache

In [10]:
(
        clean_cache,
        corrupt_cache,
        clean_logit_outputs,
        _,
        hook_points,
    ) = get_caches(model, dataloader)

Clean cache: 1it [00:04,  4.29s/it]


CLEAN CACHE COMPUTED


Corrupt cache: 1it [00:03,  3.23s/it]

CORRUPT CACHE COMPUTED


In [11]:
clean_cache

{0: TraceDict([('model.layers.0.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff93ae5d8a0>),
            ('model.layers.1.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff845278a90>),
            ('model.layers.2.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48dff0>),
            ('model.layers.3.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48e200>),
            ('model.layers.4.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48de70>),
            ('model.layers.5.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48dba0>),
            ('model.layers.6.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48d930>),
            ('model.layers.7.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48d720>),
            ('model.layers.8.self_attn.o_proj',
             <baukit.baukit.nethook.Trace at 0x7ff83d48d420>)

## Compute Value Fetcher Heads

In [ ]:
print("COMPUTING VALUE FETCHER HEADS...")
patching_scores = apply_pp(
    model=model,
    clean_cache=clean_cache,
    corrupt_cache=corrupt_cache,
    dataloader=dataloader,
    receiver_heads=[],
    receiver_layers=[],
    clean_logit_outputs=clean_logit_outputs,
    hook_points=hook_points,
    rel_pos=0,
)
torch.save(patching_scores, output_path + "value_fetcher.pt")
value_fetcher_heads = compute_topk_components(
    patching_scores=patching_scores, k=n_value_fetcher, largest=False
)
print(f"VALUE FETCHER HEADS: {value_fetcher_heads}\n")

COMPUTING VALUE FETCHER HEADS...


 38%|███▊      | 12/32 [14:31<24:15, 72.75s/it]

In [ ]:
print("Are you still here?")  

# Code LLAMA

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")
model = AutoModelForCausalLM.from_pretrained("codellama/CodeLlama-7b-Instruct-hf")